In [1]:
import csv
import re
import pandas as pd

In [2]:
abbrev_dict = {}

with open('data/Contractions (Clean).csv', mode='r') as infile:
    reader = csv.reader(infile)
    next(reader)
    abbrev_dict = {rows[0]:rows[1] for rows in reader}
    
def expandNotam(msg):
    new_string = re.sub(r"[^a-zA-Z]"," ",str(msg))
    new_msg = ''
    for wrd in new_string.split():        
        try:
            new_msg = new_msg + ' ' + abbrev_dict[wrd]
        except:
            new_msg = new_msg + ' ' + wrd
    return new_msg.lower().strip()

In [3]:
#df4 = pd.read_csv('data/notams.csv', 
#                  on_bad_lines='skip', 
#                  encoding='utf-16', 
#                  parse_dates=['POSSIBLE_START_DATE', 'POSSIBLE_END_DATE', 'ISSUE_DATE', 'CANCELED_DATE'])


df4 = pd.read_pickle("data/allData.pkl")

#df4['TEXT'] = df4['TEXT'].apply(expandNotam)

df4['MIN_ALT'] = df4['MIN_ALT'].replace('MSL', 0)
df4['MAX_ALT'] = df4['MAX_ALT'].replace('MSL', 0)
df4['MIN_ALT'] = pd.to_numeric(df4['MIN_ALT'], errors='coerce')
df4['MAX_ALT'] = pd.to_numeric(df4['MAX_ALT'], errors='coerce')

df4['POSSIBLE_START_DATE'] = pd.to_datetime(df4['POSSIBLE_START_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['POSSIBLE_END_DATE'] = pd.to_datetime(df4['POSSIBLE_END_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['ISSUE_DATE'] = pd.to_datetime(df4['ISSUE_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')
df4['CANCELED_DATE'] = pd.to_datetime(df4['CANCELED_DATE'], errors = 'coerce', format = '%Y/%m/%d %H:%M:%S')

df4.loc[df4['CANCELED_DATE'].isna(), 'CANCELED_DATE'] = df4.loc[df4['CANCELED_DATE'].isna(), 'POSSIBLE_END_DATE']
df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_END_DATE'] = df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'CANCELED_DATE']

est_days = (df4['POSSIBLE_END_DATE'] - df4['POSSIBLE_START_DATE']).mode()

df4.loc[df4['CANCELED_DATE'].isna(), 'CANCELED_DATE'] = df4.loc[df4['CANCELED_DATE'].isna(), 'POSSIBLE_START_DATE']
df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_END_DATE'] = df4.loc[df4['POSSIBLE_END_DATE'].isna(), 'POSSIBLE_START_DATE']

df4 = df4.dropna( how='all', subset=['POSSIBLE_END_DATE', 'POSSIBLE_START_DATE'])

In [4]:
df4.isna().sum()

NOTAM_REC_ID                    0
FNS_ID                          0
FILENAME                        0
NOTAM_ID                       13
NOTAM_TYPE                 138171
RELATED_NOTAM_ID           819715
SIMPLE_TEXT                517883
LOC_ID_ACCOUNTABLE_ORG     543535
NOTAM_NUMBER               545475
RELATED_NOTAM_NUMBER       969222
TEXT                            0
Q_CODE                     416159
Q_CODE_INTERPRETATION      597507
A_CODE                     321965
B_CODE                     432082
C_CODE                     525188
D_CODE                     954180
E_CODE                        237
F_CODE                     412557
G_CODE                     412498
CLASSIFICATION                 16
POSSIBLE_NOTAM_ID              13
MIN_ALT                    305058
MAX_ALT                    337613
MIN_ALT_REF_TYPE           392547
MAX_ALT_REF_TYPE           435186
POSSIBLE_START_DATE             0
POSSIBLE_END_DATE               0
ISSUE_DATE                    168
CANCELED_DATE 

In [5]:
filterA = df4[(df4['TEXT'].str.contains(r'(?:\s|^)rocket(?:\s|$)') == True) | 
           (df4['TEXT'].str.contains(r'(?:\s|^)space(?:\s|$)') == True) |
           (df4['TEXT'].str.contains(r'(?:\s|^)launch(?:\s|$)') == True)]

In [6]:
filterA

,NOTAM_REC_ID,FNS_ID,FILENAME,NOTAM_ID,NOTAM_TYPE,RELATED_NOTAM_ID,SIMPLE_TEXT,LOC_ID_ACCOUNTABLE_ORG,NOTAM_NUMBER,RELATED_NOTAM_NUMBER,...,ISSUE_DATE,CANCELED_DATE,AFFECTED_FIR,DESIGNATOR,DESIGNATOR_NAME,LOCATION_NAME,ACCOUNT_ID,LOCATION_CODE,LAUNCHES_REC_ID,TOPIC
204,205,FNS_ID_43133373,E:\Working\Sherlock_NOTAM_Data\Temporary\notam...,D4034/16,NOTAMR,D3993/16,NaN,NaN,NaN,NaN,...,2016-05-24 09:03:00,2016-05-24 12:54:00,LRBB,NaN,NaN,CIVIL AERONAUTICAL AUTHORITY,LRBBYNYX,LRBB,NaN,2
205,206,FNS_ID_43133372,E:\Working\Sherlock_NOTAM_Data\Temporary\notam...,D4033/16,NOTAMR,D3992/16,NaN,NaN,NaN,NaN,...,2016-05-24 09:02:00,2016-05-24 12:53:00,LRBB,NaN,NaN,CIVIL AERONAUTICAL AUTHORITY,LRBBYNYX,LRBB,NaN,2
206,207,FNS_ID_43133374,E:\Working\Sherlock_NOTAM_Data\Temporary\notam...,D4035/16,NOTAMR,D3994/16,NaN,NaN,NaN,NaN,...,2016-05-24 09:04:00,2016-05-24 12:54:00,LRBB,NaN,NaN,CIVIL AERONAUTICAL AUTHORITY,LRBBYNYX,LRBB,NaN,2
207,208,FNS_ID_43133370,E:\Working\Sherlock_NOTAM_Data\Temporary\notam...,D4036/16,NOTAMR,D3995/16,NaN,NaN,NaN,NaN,...,2016-05-24 09:04:00,2016-05-24 12:55:00,LRBB,NaN,NaN,CIVIL AERONAUTICAL AUTHORITY,LRBBYNYX,LRBB,NaN,2
321,322,FNS_ID_43135366,E:\Working\Sherlock_NOTAM_Data\Temporary\notam...,0431/16,NOTAMN,NaN,!GWO 05/431 IGB AIRSPACE UNMANNED ROCKET WI AN...,GWO,05/431,NaN,...,2016-05-24 13:06:00,2016-05-25 23:00:00,NaN,NaN,NaN,IGB-VORTAC,GWO,IGB,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061776,1062607,FNS_ID_57041032,E:\Working\Sherlock_NOTAM_Data\Temporary\notam...,4342/20,NaN,NaN,!FDC 0/4342 ZJX PART 1 OF 6 FL..AIRSPACE CAPE ...,FDC,0/4342,0/4342,...,2020-05-28 14:35:00,2020-05-30 12:52:00,NaN,NaN,NaN,ZJX ARTCC,FDC,ZJX,NaN,5
1061803,1062634,FNS_ID_57031884,E:\Working\Sherlock_NOTAM_Data\Temporary\notam...,A1038/20,NOTAMN,NaN,NaN,NaN,NaN,NaN,...,2020-05-27 15:59:00,2020-05-30 19:56:00,KZJX,NaN,NaN,ZJX ARTCC,KZJX,ZJX,NaN,6
1061804,1062635,FNS_ID_57031885,E:\Working\Sherlock_NOTAM_Data\Temporary\notam...,A0824/20,NOTAMN,NaN,NaN,NaN,NaN,NaN,...,2020-05-27 16:00:00,2020-05-30 19:56:00,KZMA,NaN,NaN,ZMA ARTCC,KZMA,ZMA,NaN,6
1061867,1062698,FNS_ID_56966698,E:\Working\Sherlock_NOTAM_Data\Temporary\notam...,F1513/20,NOTAMN,NaN,NaN,NaN,NaN,NaN,...,2020-05-18 16:28:00,2020-05-30 13:14:00,YMMM,NaN,NaN,MELBOURNE (FIR/FIC/ACC/COM/MET),YBBBZEZX,YMMM,NaN,6


In [7]:
from datetime import datetime

time = '10/05/2017  08:00:00 AM'

date_time_obj = datetime.strptime(time, '%m/%d/%Y %H:%M:%S %p')

In [9]:
filterB = filterA[(filterA['POSSIBLE_START_DATE'] <= time) & (filterA['POSSIBLE_END_DATE'] >= time)]

In [13]:
filterC = filterB[filterB['TOPIC'] == 6]

In [14]:
print(filterC['TEXT'])

232874    airspace reservation for cargo launch will tak...
345153    hydrogen filled balloon will be launched from ...
353624    crane erected at national aeronautics and spac...
384942    russian navy exercise rocket test firings will...
392654    flight level airspace kennedy space center fli...
392655    flight level airspace kennedy space center fli...
393535    military operations area opmil area active sub...
393536    military operations area opmil area active sub...
393537    military operations area opmil area active sub...
393583    military operations area opmil area active sub...
393587    military operations area opmil area active sub...
393661    temporary restricted area ariane zb volcanic a...
393662    temporary restricted area ariane za volcanic a...
394195    stationary altitude reservation cape air traff...
394196    temporary flight restrictions kennedy space ce...
394197    temporary flight restrictions kennedy space ce...
394204    absolute (temperature) notamc 